# Language Sampling with Advanced Filtering

Этот notebook демонстрирует возможности системы создания языковых выборок с расширенной фильтрацией.

## Функциональность:
- 📍 Фильтрация по макроареалам
- 📚 Фильтрация по языкам документации (Sources' Languages)
- 🔤 Фильтрация по WALS фичам
- 📊 Фильтрация по Grambank фичам
- ✅ Обязательное включение языков
- ❌ Исключение языков
- 🔄 4 алгоритма сэмплинга

In [1]:
# Импорты
from make_sample import GenusSample, SamplingResult
from database import (
    get_all_features, 
    get_feature_values, 
    get_all_document_language_codes,
    get_document_languages
)

print("✓ Модули загружены успешно")

✓ Модули загружены успешно


## 1. Базовая выборка без фильтров

Создаём простую выборку - один язык из каждого рода.

In [2]:
# Базовая выборка
sampler = GenusSample()
result = sampler.genus_sample()

print(f"📊 Выбрано языков: {len(result.languages)}")
print(f"📊 Охвачено родов: {len(result.included_genera)}")
print(f"\nПримеры языков:")
for lang in result.languages[:5]:
    print(f"  • {lang.name} ({lang.macroarea.name if lang.macroarea else 'Unknown'})")

📊 Выбрано языков: 2313
📊 Охвачено родов: 2313

Примеры языков:
  • Adyghe (Eurasia)
  • Siwi (Africa)
  • Mofu-Gudur (Africa)
  • Somrai (Africa)
  • Herde (Africa)


## 2. Фильтрация по макроареалам

Выбираем языки только из определённых регионов.

In [3]:
# Фильтр по макроареалам: только Африка и Евразия
sampler = GenusSample(macroareas=['Africa', 'Eurasia'])
result = sampler.primary_sample(sample_size=30)

print(f"📊 Выбрано языков: {len(result.languages)}")
macroareas = {lang.macroarea.name for lang in result.languages if lang.macroarea}
print(f"📍 Макроареалы: {macroareas}")
print(f"\nПримеры:")
for lang in result.languages[:5]:
    print(f"  • {lang.name} ({lang.macroarea.name if lang.macroarea else 'Unknown'})")

📊 Выбрано языков: 30
📍 Макроареалы: {'Africa', 'Eurasia'}

Примеры:
  • Tsotsitaal (Africa)
  • Gail (Africa)
  • Meroitic (Africa)
  • Kiong (Africa)
  • Yeni (Africa)


## 3. Фильтрация по языкам документации

Выбираем языки, для которых есть источники на определённых языках (например, на английском).

In [4]:
# Сначала посмотрим, какие языки документации доступны
doc_langs = get_all_document_language_codes()
print(f"📚 Доступно языков документации: {len(doc_langs)}")
print(f"Примеры: {sorted(doc_langs)[:10]}")

📚 Доступно языков документации: 19
Примеры: ['bel', 'cmn', 'deu', 'eng', 'fra', 'glg', 'ind', 'isl', 'ita', 'jpn']


In [5]:
# Фильтр по языку документации: только языки с источниками на английском
sampler = GenusSample(
    doc_languages=['eng'],
    macroareas=['Africa']
)
result = sampler.primary_sample(sample_size=20)

print(f"📊 Выбрано языков: {len(result.languages)}")
print(f"\nПримеры языков с английской документацией:")
for lang in result.languages[:5]:
    doc_langs = get_document_languages(lang.glottocode)
    print(f"  • {lang.name} - документация на: {doc_langs}")

📊 Выбрано языков: 4

Примеры языков с английской документацией:
  • Biafada - документация на: ['eng']
  • Coptic - документация на: ['eng']
  • Ibibio - документация на: ['eng']
  • Tegali - документация на: ['deu', 'eng']


## 4. Фильтрация по WALS фичам

Выбираем языки с определёнными лингвистическими характеристиками из WALS.

In [6]:
# Посмотрим доступные WALS фичи
wals_features = get_all_features(source='WALS')
print(f"📊 Всего WALS фичей: {len(wals_features)}")
print("\nПримеры фичей:")
for feature in wals_features[:3]:
    values = get_feature_values(feature.code)
    print(f"  • {feature.code}: {feature.name}")
    print(f"    Значений: {len(values)}")

📊 Всего WALS фичей: 192

Примеры фичей:
  • 1A: Consonant Inventories
    Значений: 5
  • 2A: Vowel Quality Inventories
    Значений: 3
  • 3A: Consonant-Vowel Ratio
    Значений: 5


In [7]:
# Фильтр по WALS: языки с SOV порядком слов
sampler = GenusSample(
    wals_features={'81A': ['1.0']},  # SOV word order
    macroareas=['Eurasia']
)
result = sampler.primary_sample(sample_size=15)

print(f"📊 Выбрано языков с SOV порядком: {len(result.languages)}")
print("\nПримеры:")
for lang in result.languages[:5]:
    print(f"  • {lang.name} ({lang.macroarea.name if lang.macroarea else 'Unknown'})")

📊 Выбрано языков с SOV порядком: 2

Примеры:
  • Wakhi (Eurasia)
  • West Itelmen (Eurasia)


## 5. Фильтрация по Grambank фичам

Выбираем языки по характеристикам из Grambank.

In [8]:
# Фильтр по Grambank: языки с артиклями
sampler = GenusSample(
    grambank_features={
        'GB020': ['1'],  # Has definite/specific articles
        'GB021': ['1']   # Has indefinite articles
    }
)
result = sampler.primary_sample(sample_size=20)

print(f"📊 Выбрано языков с артиклями: {len(result.languages)}")
print("\nПримеры:")
for lang in result.languages[:5]:
    print(f"  • {lang.name} ({lang.macroarea.name if lang.macroarea else 'Unknown'})")

📊 Выбрано языков с артиклями: 1

Примеры:
  • Tera (Africa)


## 6. Комбинированные фильтры

Можно комбинировать любые фильтры для точной настройки выборки.

In [9]:
# Комплексный фильтр: Евразия + английская документация + SOV + артикли
sampler = GenusSample(
    macroareas=['Eurasia'],
    doc_languages=['eng'],
    wals_features={'81A': ['1.0']},  # SOV
    grambank_features={'GB020': ['1']}  # Has articles
)
result = sampler.genus_sample()

print(f"📊 Выбрано языков с комбинированными фильтрами: {len(result.languages)}")
print("\nПримеры:")
for lang in result.languages[:10]:
    print(f"  • {lang.name} - {lang.genus.name if lang.genus else 'Unknown genus'}")

📊 Выбрано языков с комбинированными фильтрами: 13

Примеры:
  • Abkhaz - Northwest Caucasian
  • Lishanid Noshan - Semitic
  • Odia - Indic
  • Western Farsi - Iranian
  • Bunan - Bodic
  • Nocte Naga - Brahmaputran
  • Lepcha - Himalayish
  • Angami Naga - Angami-Pochuri
  • Mizo - Kuki-Chin
  • East-Central Tangkhul Naga - Tangkhul-Maring


## 7. Включение и исключение языков

Можно обязательно включить определённые языки или исключить их из выборки.

In [10]:
# Обязательное включение определённых языков
sampler = GenusSample(
    include_languages=['stan1293', 'russ1263'],  # Русский и какой-то другой
    exclude_languages=['fren1241'],  # Исключить французский
    macroareas=['Eurasia']
)
result = sampler.primary_sample(sample_size=20)

glottocodes = [lang.glottocode for lang in result.languages]
print(f"📊 Выбрано языков: {len(result.languages)}")
print(f"✓ stan1293 включён: {'stan1293' in glottocodes}")
print(f"✓ russ1263 включён: {'russ1263' in glottocodes}")
print(f"✗ fren1241 исключён: {'fren1241' not in glottocodes}")

📊 Выбрано языков: 20
✓ stan1293 включён: True
✓ russ1263 включён: True
✗ fren1241 исключён: True


## 8. Различные алгоритмы сэмплинга

Доступны 4 алгоритма с разными стратегиями отбора.

In [11]:
# Сравнение разных алгоритмов
sampler = GenusSample(macroareas=['Africa'])

# 1. genus_sample() - один язык из каждого рода
gs = sampler.genus_sample()
print(f"1️⃣ Genus Sample: {len(gs.languages)} языков из {len(gs.included_genera)} родов")

# 2. core_sample() - языки с пригодными источниками
cs = sampler.core_sample()
print(f"2️⃣ Core Sample: {len(cs.languages)} языков из {len(cs.included_genera)} родов")

# 3. restricted_sample() - с пропорциональным распределением
rs = sampler.restricted_sample()
print(f"3️⃣ Restricted Sample: {len(rs.languages)} языков из {len(rs.included_genera)} родов")

# 4. primary_sample() - фиксированный размер
ps = sampler.primary_sample(sample_size=30)
print(f"4️⃣ Primary Sample (size=30): {len(ps.languages)} языков из {len(ps.included_genera)} родов")

1️⃣ Genus Sample: 510 языков из 510 родов
2️⃣ Core Sample: 510 языков из 510 родов
3️⃣ Restricted Sample: 510 языков из 510 родов
4️⃣ Primary Sample (size=30): 30 языков из 30 родов


## 9. Расширенная выборка (Extended Sample)

Можно объединять несколько выборок в одну расширенную.

In [12]:
# Создаём две выборки и объединяем их
sampler1 = GenusSample(macroareas=['Africa'])
sample1 = sampler1.primary_sample(sample_size=10)

sampler2 = GenusSample(macroareas=['Eurasia'])
sample2 = sampler2.primary_sample(sample_size=10)

# Объединяем
extended = sample1.extend_sample(sample2)

print(f"📊 Выборка 1 (Africa): {len(sample1.languages)} языков")
print(f"📊 Выборка 2 (Eurasia): {len(sample2.languages)} языков")
print(f"📊 Расширенная выборка: {len(extended.languages)} языков из {len(extended.included_genera)} родов")

📊 Выборка 1 (Africa): 10 языков
📊 Выборка 2 (Eurasia): 10 языков
📊 Расширенная выборка: 20 языков из 20 родов


## Заключение

Система сэмплинга предоставляет гибкие инструменты для создания языковых выборок с точной настройкой через множество фильтров:

✅ **Географические фильтры**: макроареалы  
✅ **Документальные фильтры**: языки источников  
✅ **Лингвистические фильтры**: WALS и Grambank фичи  
✅ **Ручная настройка**: включение/исключение конкретных языков  
✅ **4 алгоритма**: от простого до пропорционального распределения  

Все фильтры можно комбинировать для максимально точной выборки!